In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
!nvidia-smi
os.chdir("../")
from parc.model import model_burgers as model
import parc.data.data_burgers as data
!nvidia-smi


2024-05-27 11:35:51.068140: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-27 11:35:53.189780: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/:/usr/local/cuda/include/:/usr/local/cuda/lib/:/usr/local/cuda/lib64/:/usr/lib/x86_64-linux-gnu/:/opt/apps/TensorRT-8.6.1.6/lib/:/opt/apps/TensorRT-8.6.1.6/include/:/usr/include/x86_64-linux-gnu/:/usr/include/
2024-05-27 11:35:53.190468: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvi

Mon May 27 11:35:55 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A30                     On  |   00000000:21:00.0 Off |                    0 |
| N/A   30C    P0             28W /  165W |       0MiB /  24576MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

2024-05-27 11:35:56.647821: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-27 11:35:56.772806: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22280 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 0000:21:00.0, compute capability: 8.0


Mon May 27 11:35:57 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A30                     On  |   00000000:21:00.0 Off |                    0 |
| N/A   31C    P0             48W /  165W |   22515MiB /  24576MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
burgers_data = data.DataBurgers()
R_list = [7500]
a_list = [0.9]
w_list = [0.8]
clipped_data = burgers_data.clip_raw_data(R_list,a_list,w_list, purpose = 'training')

# Validation

In [ ]:
R_list_test = [3000]
a_list_test = [1.00]
w_list_test = [0.85]
clipped_data_test = burgers_data.clip_raw_data( R_list_test, a_list_test, w_list_test, sequence_length = 100, purpose = 'validation')

## Load model from weight files

In [ ]:
tf.keras.backend.clear_session()
parc = model.PARCv2_burgers(n_time_step = 100, step_size = 1/100, solver = "heun")
parc.differentiator.load_weights('./Pretrained_Weights/burgers/parc2_diff_burgers_heun.h5')
parc.integrator.load_weights('./Pretrained_Weights/burgers/parc2_int_burgers_heun.h5')

## Make prediction on the test set

In [ ]:
prediction_data = []
for j in range(1):
    input_seq_current = tf.cast(clipped_data_test[j:j+1,:,:,:3], dtype = tf.float32)
    res = parc.predict(input_seq_current)
    prediction_data.append(res)
    print('Finish case ', j)
prediction_data = np.concatenate(prediction_data, axis = 0)

In [ ]:
#np.save('./results/burgers/parcv2_burgers.npy',prediction_data)

In [ ]:
print(prediction_data.shape)
for i in range(75):
    plt.imshow(prediction_data[0,:,:,(i)+3])
    plt.show()